In [1]:
import os
#import numpy for number array handling and represent rgb image pixel values
import numpy as np

#import tensorflow to use any tools needed for deep learning
import tensorflow as tf

#import keras api needed to implement deep learning techiques
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPool2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#import libraries for visualization of data
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

#Allow charts and graphics to display right below the page of browser setup
%matplotlib inline

**Download data**

In [2]:
#paths to the train, validation and test image datasets 
train_path = '../input/garbage-classification/garbage classification/Garbage classification'
valid_path = '../input/garbage-classification/garbage classification/Garbage classification'

**Set up for Generic CNN model**

In [3]:
labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

for label in labels:
    directory = os.path.join(train_path, label)
    print("Images of label \"" + label + "\":\t", len(os.listdir(directory)))

In [4]:
# plotting images of different review for understanding the dataset

plt.figure(figsize=(30,14))

for i in range(6):
    directory = os.path.join(train_path, labels[i])
    for j in range(10):
        path = os.path.join(directory, os.listdir(directory)[j])
        img = mpimg.imread(path)
        
        plt.subplot(6, 10, i*10 + j + 1)
        plt.imshow(img)
        
        if j == 0:
            plt.ylabel(labels[i], fontsize=20)
        
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);
plt.tight_layout()
plt.show()

In [5]:
# checking size of individual image

directory = os.path.join(train_path, 'cardboard')
path = os.path.join(directory, os.listdir(directory)[0])
image = mpimg.imread(path)
image.shape

In [6]:
# creating the model

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(384, 512, 3)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),
    
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),
    
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),

  tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.2),
    
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(6, activation='softmax')
])

model.summary()

In [7]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = 0.0001), metrics=['accuracy'])

In [8]:
# creating generators for combining data and increasing the gainable insights by slightly modifying the images in the dataset

train_datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,
                                   rotation_range=15,zoom_range=0.1,
                                   width_shift_range=0.15,height_shift_range=0.15,
                                   shear_range=0.1,
                                   fill_mode="nearest",
                                   rescale=1./255., 
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(train_path, target_size=(384, 512), batch_size=32, class_mode='binary', subset='training')
validation_generator = train_datagen.flow_from_directory(train_path, target_size=(384, 512), batch_size=32, class_mode='binary', subset='validation')

In [9]:
history = model.fit(train_generator, epochs=50, verbose=1, validation_data=validation_generator)

In [10]:
num_epochs = range(1,51)
plt.plot(num_epochs, history.history['accuracy'], label='Training', c='r')
plt.plot(num_epochs, history.history['val_accuracy'], label='Validation', c='b')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [11]:
plt.plot(num_epochs,history.history['loss'],c="red",label="Training")
plt.plot(num_epochs,history.history['val_loss'],c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

**Generate training data set**

In [12]:
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2).flow_from_directory(
    directory=train_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], 
                                                        batch_size=16, subset='training')

**Generating validatoin data set**

In [13]:
valid_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    validation_split=0.2).flow_from_directory(
    directory=valid_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='validation')

**View the preprocessed picture**

In [14]:
def plotImages(images):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img.astype(np.uint8))
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [15]:
imgs, labels = next(train_batches)
plotImages(imgs)

**Build a convolutional neural network**

In [16]:
#imput image size
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
#Import the pretrained weights VGG basic model
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')

In [17]:
base_model.summary()

**Add layers to the model and build the model**

In [18]:

model = Sequential()

base_model.trainable=False

#Load the pretrained model
model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=512, activation='relu'))  
model.add(BatchNormalization())                 
model.add(Dropout(0.2))                         

model.add(Dense(units=128, activation='relu')) 
model.add(BatchNormalization())                
model.add(Dropout(0.2))                        

model.add(Dense(units=6, activation='softmax')) 
model.summary()

**Set the loss functions、metrics、adam（The initial learning rate 0.0001）**

In [19]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model_details = model.fit(x=train_batches, validation_data=valid_batches, epochs=20, verbose=2)

In [21]:
loss = model_details.history['loss']
validation_loss = model_details.history['val_loss']

In [22]:
accuracy = model_details.history['accuracy']
validation_accuracy = model_details.history['val_accuracy']

In [23]:
len(validation_accuracy)

**Fine-tune model**

In [24]:
base_model.trainable=True

In [25]:
model.summary()

In [26]:
model_details = model.fit(x=train_batches,validation_data=valid_batches,
                          epochs=8, verbose=2)

In [27]:
model.save('vgg16.h5')

**View training results**

In [28]:
loss.extend(model_details.history['loss'])
validation_loss.extend(model_details.history['val_loss'])
accuracy.extend(model_details.history['accuracy'])
validation_accuracy.extend(model_details.history['val_accuracy'])

In [29]:
len(validation_accuracy)

In [30]:
epochs = range(1, len(loss)+1)

fig1 = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training")
plt.plot(epochs,validation_loss,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(epochs)
plt.legend()

In [31]:
epochs = range(1, len(accuracy)+1)

fig2 = plt.figure(figsize=(10,6))
plt.plot(epochs,accuracy,c="red",label="Training")
plt.plot(epochs,validation_accuracy,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(epochs)
plt.legend()

**RESNET50**

In [32]:
##RESNET50

In [33]:
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2).flow_from_directory(
    directory=train_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='training')

In [34]:
valid_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2).flow_from_directory(
    directory=valid_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='validation')

In [35]:
def plotImages(images):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img.astype(np.uint8))
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [36]:
imgs, labels = next(train_batches)
plotImages(imgs)

In [37]:
#imput image size
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
#Import the pretrained weights VGG basic model
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')

In [38]:
base_model.summary()

In [39]:
model = Sequential()

base_model.trainable=False

#Load the pretrained model
model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=512, activation='relu'))  
model.add(BatchNormalization())                 
model.add(Dropout(0.2))                         

model.add(Dense(units=128, activation='relu')) 
model.add(BatchNormalization())                
model.add(Dropout(0.2))                        

model.add(Dense(units=6, activation='softmax')) 
model.summary()

In [40]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
model_details = model.fit(x=train_batches, validation_data=valid_batches, epochs=20, verbose=2)

In [42]:
loss = model_details.history['loss']
validation_loss = model_details.history['val_loss']

In [43]:
accuracy = model_details.history['accuracy']
validation_accuracy = model_details.history['val_accuracy']

In [44]:
base_model.trainable=True

In [45]:
model.summary()

In [46]:
model_details = model.fit(x=train_batches,validation_data=valid_batches,
                          epochs=8, verbose=2)

In [47]:
model.save('ResNet50.h5')

In [48]:
loss.extend(model_details.history['loss'])
validation_loss.extend(model_details.history['val_loss'])
accuracy.extend(model_details.history['accuracy'])
validation_accuracy.extend(model_details.history['val_accuracy'])

In [49]:
epochs = range(1, len(loss)+1)

fig1 = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training")
plt.plot(epochs,validation_loss,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(epochs)
plt.legend()

In [50]:
epochs = range(1, len(accuracy)+1)

fig2 = plt.figure(figsize=(10,6))
plt.plot(epochs,accuracy,c="red",label="Training")
plt.plot(epochs,validation_accuracy,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(epochs)
plt.legend()

**VGG19**

In [51]:
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2).flow_from_directory(
    directory=train_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='training')




valid_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input,
    validation_split=0.2).flow_from_directory(
    directory=valid_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='validation')


def plotImages(images):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img.astype(np.uint8))
        ax.axis('off')
    plt.tight_layout()
    plt.show()


imgs, labels = next(train_batches)
plotImages(imgs)


#imput image size
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
#Import the pretrained weights VGG basic model
base_model = tf.keras.applications.VGG19(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')


base_model.summary()


model = Sequential()

base_model.trainable=False

#Load the pretrained model
model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=512, activation='relu'))  
model.add(BatchNormalization())                 
model.add(Dropout(0.2))                         

model.add(Dense(units=128, activation='relu')) 
model.add(BatchNormalization())                
model.add(Dropout(0.2))                        

model.add(Dense(units=6, activation='softmax')) 
model.summary()


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model_details = model.fit(x=train_batches, validation_data=valid_batches, epochs=20, verbose=2)

loss = model_details.history['loss']
validation_loss = model_details.history['val_loss'] 


accuracy = model_details.history['accuracy']
validation_accuracy = model_details.history['val_accuracy']

base_model.trainable=True

model.summary()

model_details = model.fit(x=train_batches,validation_data=valid_batches,
                          epochs=8, verbose=2)


model.save('vgg19.h5')


loss.extend(model_details.history['loss'])
validation_loss.extend(model_details.history['val_loss'])
accuracy.extend(model_details.history['accuracy'])
validation_accuracy.extend(model_details.history['val_accuracy'])


print("Epochs vs Validation Graph")
epochs = range(1, len(loss)+1)

fig1 = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training")
plt.plot(epochs,validation_loss,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(epochs)
plt.legend()

print("Epochs vs Accuracy Graph")
epochs = range(1, len(accuracy)+1)

fig2 = plt.figure(figsize=(10,6))
plt.plot(epochs,accuracy,c="red",label="Training")
plt.plot(epochs,validation_accuracy,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(epochs)
plt.legend()

**MobileNet**

In [52]:
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2).flow_from_directory(
    directory=train_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='training')




valid_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet.preprocess_input,
    validation_split=0.2).flow_from_directory(
    directory=valid_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='validation')


def plotImages(images):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img.astype(np.uint8))
        ax.axis('off')
    plt.tight_layout()
    plt.show()


imgs, labels = next(train_batches)
plotImages(imgs)


#imput image size
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
#Import the pretrained weights VGG basic model
base_model = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')


base_model.summary()


model = Sequential()

base_model.trainable=False

#Load the pretrained model
model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=512, activation='relu'))  
model.add(BatchNormalization())                 
model.add(Dropout(0.2))                         

model.add(Dense(units=128, activation='relu')) 
model.add(BatchNormalization())                
model.add(Dropout(0.2))                        

model.add(Dense(units=6, activation='softmax')) 
model.summary()


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model_details = model.fit(x=train_batches, validation_data=valid_batches, epochs=20, verbose=2)

loss = model_details.history['loss']
validation_loss = model_details.history['val_loss'] 


accuracy = model_details.history['accuracy']
validation_accuracy = model_details.history['val_accuracy']

base_model.trainable=True

model.summary()

model_details = model.fit(x=train_batches,validation_data=valid_batches,
                          epochs=8, verbose=2)


model.save('vgg19.h5')


loss.extend(model_details.history['loss'])
validation_loss.extend(model_details.history['val_loss'])
accuracy.extend(model_details.history['accuracy'])
validation_accuracy.extend(model_details.history['val_accuracy'])


print("Epochs vs Validation loss graph")
epochs = range(1, len(loss)+1)

fig1 = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training")
plt.plot(epochs,validation_loss,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(epochs)
plt.legend()

print("Epochs vs Accuracy Graph")
epochs = range(1, len(accuracy)+1)

fig2 = plt.figure(figsize=(10,6))
plt.plot(epochs,accuracy,c="red",label="Training")
plt.plot(epochs,validation_accuracy,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(epochs)
plt.legend()

In [53]:
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.densenet.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2).flow_from_directory(
    directory=train_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='training')




valid_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.densenet.preprocess_input,
    validation_split=0.2).flow_from_directory(
    directory=valid_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='validation')


def plotImages(images):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img.astype(np.uint8))
        ax.axis('off')
    plt.tight_layout()
    plt.show()


imgs, labels = next(train_batches)
plotImages(imgs)


#imput image size
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
#Import the pretrained weights Densenet121 basic model
base_model = tf.keras.applications.DenseNet121(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')


base_model.summary()


model = Sequential()

base_model.trainable=False

#Load the pretrained model
model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=512, activation='relu'))  
model.add(BatchNormalization())                 
model.add(Dropout(0.2))                         

model.add(Dense(units=128, activation='relu')) 
model.add(BatchNormalization())                
model.add(Dropout(0.2))                        

model.add(Dense(units=6, activation='softmax')) 
model.summary()


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model_details = model.fit(x=train_batches, validation_data=valid_batches, epochs=20, verbose=2)

loss = model_details.history['loss']
validation_loss = model_details.history['val_loss'] 


accuracy = model_details.history['accuracy']
validation_accuracy = model_details.history['val_accuracy']

base_model.trainable=True

model.summary()

model_details = model.fit(x=train_batches,validation_data=valid_batches,
                          epochs=8, verbose=2)


model.save('DenseNet121.h5')


loss.extend(model_details.history['loss'])
validation_loss.extend(model_details.history['val_loss'])
accuracy.extend(model_details.history['accuracy'])
validation_accuracy.extend(model_details.history['val_accuracy'])

print("Epochs vs Validation loss graph")
epochs = range(1, len(loss)+1)

fig1 = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training")
plt.plot(epochs,validation_loss,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(epochs)
plt.legend()

print("Epochs vs Accuracy Graph")
epochs = range(1, len(accuracy)+1)

fig2 = plt.figure(figsize=(10,6))
plt.plot(epochs,accuracy,c="red",label="Training")
plt.plot(epochs,validation_accuracy,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(epochs)
plt.legend()

In [54]:
train_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.xception.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2).flow_from_directory(
    directory=train_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='training')




valid_batches = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.xception.preprocess_input,
    validation_split=0.2).flow_from_directory(
    directory=valid_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='validation')


def plotImages(images):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img.astype(np.uint8))
        ax.axis('off')
    plt.tight_layout()
    plt.show()


imgs, labels = next(train_batches)
plotImages(imgs)


#imput image size
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
#Import the pretrained weights VGG basic model
base_model = tf.keras.applications.Xception(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')


base_model.summary()


model = Sequential()

base_model.trainable=True

#Load the pretrained model
model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=512, activation='relu'))  
model.add(BatchNormalization())                 
model.add(Dropout(0.2))                         

model.add(Dense(units=128, activation='relu')) 
model.add(BatchNormalization())                
model.add(Dropout(0.2))                        

model.add(Dense(units=6, activation='softmax')) 
model.summary()


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model_details = model.fit(x=train_batches, validation_data=valid_batches, epochs=20, verbose=2)

loss = model_details.history['loss']
validation_loss = model_details.history['val_loss'] 


accuracy = model_details.history['accuracy']
validation_accuracy = model_details.history['val_accuracy']

print("Epochs vs Validation loss graph")
epochs = range(1, len(loss)+1)

fig1 = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training")
plt.plot(epochs,validation_loss,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(epochs)
plt.legend()

print("Epochs vs Accuracy Graph")
epochs = range(1, len(accuracy)+1)

fig2 = plt.figure(figsize=(10,6))
plt.plot(epochs,accuracy,c="red",label="Training")
plt.plot(epochs,validation_accuracy,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(epochs)
plt.legend()

In [55]:
### Mobilenet

In [56]:
train_batches = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.1,
    horizontal_flip=True,
    vertical_flip=True).flow_from_directory(
    directory=train_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='training')




valid_batches = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1).flow_from_directory(
    directory=valid_path, target_size=(224,224), classes=['cardboard', 'glass', 'metal', 
                                                         'paper', 'plastic', 'trash'], batch_size=16, subset='validation')


def plotImages(images):
    fig, axes = plt.subplots(1, 6, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img.astype(np.uint8))
        ax.axis('off')
    plt.tight_layout()
    plt.show()


imgs, labels = next(train_batches)
plotImages(imgs)


#imput image size
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
#Import the pretrained weights VGG basic model
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, 
                                        include_top=False,
                                        weights='imagenet')


base_model.summary()


model = Sequential()

base_model.trainable=False

#Load the pretrained model
model.add(base_model)

model.add(GlobalAveragePooling2D())

model.add(Dense(units=512, activation='relu'))  
model.add(BatchNormalization())                 
model.add(Dropout(0.2))                         

model.add(Dense(units=128, activation='relu')) 
model.add(BatchNormalization())                
model.add(Dropout(0.2))                        

model.add(Dense(units=6, activation='softmax')) 
model.summary()


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model_details = model.fit(x=train_batches, validation_data=valid_batches, epochs=20, verbose=2)

loss = model_details.history['loss']
validation_loss = model_details.history['val_loss'] 


accuracy = model_details.history['accuracy']
validation_accuracy = model_details.history['val_accuracy']

base_model.trainable=True

model.summary()

model_details = model.fit(x=train_batches,validation_data=valid_batches,
                          epochs=8, verbose=2)


model.save('DenseNet121.h5')


loss.extend(model_details.history['loss'])
validation_loss.extend(model_details.history['val_loss'])
accuracy.extend(model_details.history['accuracy'])
validation_accuracy.extend(model_details.history['val_accuracy'])

print("Epochs vs Validation Loss graph")
epochs = range(1, len(loss)+1)

fig1 = plt.figure(figsize=(10,6))
plt.plot(epochs,loss,c="red",label="Training")
plt.plot(epochs,validation_loss,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(epochs)
plt.legend()

print("Epochs vs Accuracy graph")
epochs = range(1, len(accuracy)+1)

fig2 = plt.figure(figsize=(10,6))
plt.plot(epochs,accuracy,c="red",label="Training")
plt.plot(epochs,validation_accuracy,c="blue",label="Validation")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(epochs)
plt.legend()